In [1]:
import csv
f = open('fleiss.csv')
reader = csv.reader(f)
codes = list(reader)

In [2]:
_quoteDict = {}
for codedQuote in codes: 
    if codedQuote[0] != 'Index': 
        quote = {}
        quote["quote"] = codedQuote[1]
        quote["sasha_raw"] = set(codedQuote[2].strip().split(" "))
        quote["amey_raw_add"] = set(codedQuote[3].strip().split(" "))
        quote["amey_raw_sub"] = set(codedQuote[4].strip().split(" "))
        quote["chin_raw_add"] = set(codedQuote[5].strip().split(" "))
        quote["chin_raw_sub"] = set(codedQuote[6].strip().split(" "))
        _quoteDict[int(codedQuote[0])] = quote

In [3]:
from copy import deepcopy
all_codes = set({})
quoteDict = {}
for ix, coded in _quoteDict.items(): 
    quote = {}
    quote["quote"] = coded["quote"]
    quote["sasha"] = coded["sasha_raw"]
    temp_chin = deepcopy(coded["sasha_raw"])
    temp_chin.update(coded['chin_raw_add'])
    temp_chin.difference(coded['chin_raw_sub'])
    quote['chin'] = temp_chin
    temp_amey = deepcopy(coded["sasha_raw"])
    temp_amey.update(coded['amey_raw_add'])
    temp_amey.difference(coded['amey_raw_sub'])
    quote['amey'] = temp_amey
    quoteDict[ix] = quote
    all_codes.update(quote["sasha"])
    all_codes.update(quote["chin"])
    all_codes.update(quote["amey"])
all_codes = all_codes.difference({''})

In [4]:
# all_codes

In [5]:
# Preparing codes for NLTK
from collections import defaultdict

codeDict = defaultdict(dict)
for code in all_codes: 
    for ix, quote in quoteDict.items(): 
        quotes = {}
        quotes['sasha'] = 1 if code in quote['sasha'] else 0
        quotes['chin'] = 1 if code in quote['chin'] else 0 
        quotes['amey'] = 1 if code in quote['amey'] else 0 
        quotes['quote'] = quote['quote']
        codeDict[code][ix] = quotes

In [6]:
# codeDict["#LackingSupport"]

In [24]:
# Provide a single tag, or multiple tags. 
raters = ['sasha', 'chin', 'amey']
raterFlag = [False] * len(raters)

def getAgreementCodes(tags):
    data = []
    for rater in raters: 
        for itemlabel, quote in quoteDict.items():
            if len(tags) > 1 and all([tag in quote[rater] for tag in tags]):
                data.append([rater, itemlabel, "#Both"])
            if all([tag not in quote[rater] for tag in tags]):
                data.append([rater, itemlabel, "#None"])
            else: 
                for tag in tags:
                    if tag in quote[rater]:
                        data.append([rater, itemlabel, tag])
    return data

In [25]:
labels = [
    ("#LackingSupport", "#ReceivingSupport"),
    ("#Believable", "#NotBelievable"),
    ("#MiddleGround", "#Disagreement"),
    ("#StandingGround", "#ChangedOpinion"),
    ("#Expected", "#Unexpected"),
    ("#DecreasingCertainty", "#IncreasingCertainty"),
    ("#UncertainMindsChange", "#ChangedDespiteCertainty"),
    ("#InfluenceGroup",),
    ("#InfluenceIndividual",),
    ("#InfluenceArticle",),
    ("#ReinforcedViews",),
    ("#NPCMentionedUnprompted",),
    ("#UsedPoliticalAffiliationSteriotype",),
    ("#IdentifyingSimilarGroups",),
    ("#SimilarViewsConverge",),
    ("#OpinionAttitudeDifference",),
    ("#CertaintyConvinces",),
    ("#PoliticalIdentity",),
    ("#NotEnoughInfo",),
    ("#EmotionalResponse",),
    ("#InferFactsFrom",),
    ("#CausalInference",),
    ("#Meta",),
    ("#GroupsStandGround",),
    ("#NoAnswer",),
    ("#Polarization",),
    ("#Infer",),
    ("#AgreeWithMajority",),
    ("#ExtremeChangesInOpinion",),
    ("#OpinionAttitudeAlignment",),
    ("#LimitedKnowledge",),
    ("#OvertonWindow",)
]

In [29]:
from nltk.metrics.agreement import AnnotationTask
from nltk.probability import FreqDist

total_fleiss = []
total_kappa = []
total_alpha = []
total_scotts = []

for code in labels: 
    nltk_code_format = getAgreementCodes(code)
    task = AnnotationTask(data=nltk_code_format)
    label_freqs = FreqDist(x['labels'] for x in task.data)
    print("Codes: ", label_freqs.keys())
    try: 
        print("kappa " +str(task.kappa()))
        total_kappa.append(task.kappa())
        print("fleiss " + str(task.multi_kappa()))
        total_fleiss.append(task.multi_kappa())
        print("alpha " +str(task.alpha()))
        total_alpha.append(task.alpha())
        print("scotts " + str(task.pi()))
        total_scotts.append(task.pi())
        
    except ZeroDivisionError as e: 
        print("Expected Agreement: 1.0 - ZeroDivisionError")
    
    print("fleiss: ", sum(total_fleiss)/len(total_fleiss))
    print("kappa: ", sum(total_kappa)/len(total_kappa))
    print("alpha: ", sum(total_alpha)/len(total_alpha))
    print("scotts: ", sum(total_scotts)/len(total_scotts))

Codes:  dict_keys(['#None', '#LackingSupport', '#ReceivingSupport'])
kappa 0.8592132505175983
fleiss 0.8495575221238946
alpha 0.8508124076809453
scotts 0.8493353028064998
fleiss:  0.8495575221238946
kappa:  0.8592132505175983
alpha:  0.8508124076809453
scotts:  0.8493353028064998
Codes:  dict_keys(['#Believable', '#None', '#NotBelievable'])
kappa 0.9676652401331429
fleiss 0.9674952198852773
alpha 0.9678036340452661
scotts 0.9674848581447244
fleiss:  0.9085263710045859
kappa:  0.9134392453253706
alpha:  0.9093080208631057
scotts:  0.9084100804756121
Codes:  dict_keys(['#None', '#Disagreement', '#MiddleGround'])
kappa 1.0
fleiss 1.0
alpha 1.0
scotts 1.0
fleiss:  0.939017580669724
kappa:  0.9422928302169137
alpha:  0.9395386805754038
scotts:  0.9389400536504081
Codes:  dict_keys(['#ChangedOpinion', '#Both', '#StandingGround', '#None'])
kappa 0.9631436314363144
fleiss 0.9632829373650109
alpha 0.7728329072739949
scotts 0.9632697155203458
fleiss:  0.9450839198435457
kappa:  0.947505530521763